In [1]:
from load import *
import load
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import os
import torch

In [2]:
g=load.load_random_hetero_graph('../data/PAAD_network', Specified_quantity={'lncrna': 1500, 'mirna': 168, 'mrna': 2519})
adjacency_matrix=torch.from_numpy(np.array(nx.adjacency_matrix(dgl.to_homogeneous(g).to_networkx().to_undirected()).todense(),dtype=float))
gragh = nx.Graph()
for i in range(len(adjacency_matrix)):
    gragh.add_node(i)
    gragh.nodes[i]["data"] = i

for i in range(len(adjacency_matrix)):
    for j in range(len(adjacency_matrix)):
        if adjacency_matrix[i,j]:
            gragh.add_edge(i,j)

Graph(num_nodes={'lncrna': 1500, 'mirna': 168, 'mrna': 2519},
      num_edges={('lncrna', 'lnc_lnc', 'lncrna'): 12654, ('lncrna', 'lnc_mi', 'mirna'): 186, ('lncrna', 'm_lnc', 'mrna'): 23276, ('mirna', 'mi_lnc', 'lncrna'): 186, ('mirna', 'mi_m', 'mrna'): 17320, ('mirna', 'mi_mi', 'mirna'): 132, ('mrna', 'lnc_m', 'lncrna'): 23276, ('mrna', 'm_m', 'mrna'): 60230, ('mrna', 'm_mi', 'mirna'): 17320},
      metagraph=[('lncrna', 'lncrna', 'lnc_lnc'), ('lncrna', 'mirna', 'lnc_mi'), ('lncrna', 'mrna', 'm_lnc'), ('mirna', 'lncrna', 'mi_lnc'), ('mirna', 'mrna', 'mi_m'), ('mirna', 'mirna', 'mi_mi'), ('mrna', 'lncrna', 'lnc_m'), ('mrna', 'mrna', 'm_m'), ('mrna', 'mirna', 'm_mi')])


In [3]:
rna_num = {}
num_rna = {}
curnode = 0
for name in g.ntypes:
    tmp={}
    for i in range(g.number_of_nodes(name)):
        tmp[i]=curnode
        num_rna[curnode]=(name,i)
        curnode+=1
    rna_num[name]=tmp

In [4]:
#去掉真实网络中去点度为0的孤立节点
new_nodes = [_ for _, v in nx.degree(gragh) if v != 0]
# plt.subplot(211)
graph_new = gragh.subgraph(new_nodes)
# nx.draw_networkx(graph_new, with_labels=True,node_size=50,font_size=8)
# plt.show()

In [5]:
#对联通子图进行判断
# N = 50
N = 400
# N = 400
# idex=12
CS = []
CS_len = []
init_graph = []
for c in nx.connected_components(graph_new):
    # 得到不连通的子集
    nodeSet = list(graph_new.subgraph(c).nodes())
    CS.append(c)
    CS_len.append(len(nodeSet))
    if len(nodeSet) <N:
        init_graph.append(nodeSet)
print(CS_len)
graph_new = graph_new.subgraph(CS[0])

[3164, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2]


In [6]:
#保存被删除的节点
# delete_ndoes = []
# for nodes in CS[1:]:
#     for node in nodes:
#         delete_ndoes.append(node)
# print(delete_ndoes)
# torch.save(delete_ndoes,"delete_nodes.pt")

In [7]:
# print(len(delete_ndoes))

In [8]:
degree0_nodes = list(graph_new.nodes())
lncnum = 0
minum = 0
mnum = 0
for n in degree0_nodes:
    if num_rna[n][0] == "lncrna":
        lncnum+=1
    elif num_rna[n][0] == "mirna":
        minum+=1
    elif num_rna[n][0] == "mrna":
        mnum+=1
print(lncnum,minum,mnum)

643 168 2353


In [9]:
print(len(graph_new))
nodesss = list(graph_new.nodes())
graphid_2_degree0 = torch.zeros((len(nodesss),1))
for i in range(len(nodesss)):
    graphid_2_degree0[i,0] = torch.tensor(nodesss[i])
torch.save(graphid_2_degree0,"graphid_2_degree0_"+str(N)+"_"+str(11)+"_add_ep"+".pt")

3164


In [10]:
aaa = torch.load("graphid_2_degree0_"+str(N)+"_"+str(11)+"_add_ep"+".pt")
# print(aaa[0,0])

In [11]:
print(len(graph_new.nodes()))
#获取新图的节点
new_sort_nodes = list(graph_new.nodes())
node_dict = {}
node_dict_reverse = {}
#记录新节点在邻接矩阵中的index下标
for i,n in enumerate(new_sort_nodes):
    node_dict[n] = i
    node_dict_reverse[i] = n
new_A=np.array(nx.adjacency_matrix(graph_new).todense())
#改变邻接矩阵
print(len(new_A))

3164
3164


In [12]:
# # 构建新的子图
# lnc_lnc = []
# lnc_mi = []
# lnc_m = []
# mi_lnc = []
# mi_mi = []
# mi_m = []
# m_lnc = []
# m_mi = []
# m_m = []
# sub1 = new_sort_nodes
# rna_num = [0,0,0]
# node_type_id = {}
# lnc_num = 0
# mi_num = 0
# m_num = 0
# 
# for f,node in enumerate(sub1):
#     type = num_rna[node][0]
# 
#     if type == "lncrna":
#         node_type_id[node] = (type,lnc_num)
#         lnc_num += 1
#     elif type == "mirna":
#         node_type_id[node] = (type,mi_num)
#         mi_num += 1
#     elif type == "mrna":
#         node_type_id[node] = (type,m_num)
#         m_num += 1
# 
# print(lnc_num,mi_num,m_num,end=" ")
# for index1,node in enumerate(sub1):
#     if num_rna[node][0] == "lncrna":
#         #判断邻居属性
#         for index,node1 in enumerate(sub1):
#             #子图内的邻居节点node,i
#             if new_A[node_dict[node],node_dict[node1]]==1:
#                 if num_rna[node1][0] == "lncrna":
#                     lnc_lnc.append([node_type_id[node][1],node_type_id[node1][1]])
#                 elif num_rna[node1][0] == "mirna":
#                     lnc_mi.append([node_type_id[node][1],node_type_id[node1][1]])
#                 elif num_rna[node1][0] == "mrna":
#                     lnc_m.append([node_type_id[node][1],node_type_id[node1][1]])
#     elif num_rna[node][0] == "mirna":
#         for index,node1 in enumerate(sub1):
#             #子图内的邻居节点node,i
#             if new_A[node_dict[node],node_dict[node1]]==1:
#                 if num_rna[node1][0] == "lncrna":
#                     mi_lnc.append([node_type_id[node][1],node_type_id[node1][1]])
#                 elif num_rna[node1][0] == "mirna":
#                     mi_mi.append([node_type_id[node][1],node_type_id[node1][1]])
#                 elif num_rna[node1][0] == "mrna":
#                     mi_m.append([node_type_id[node][1],node_type_id[node1][1]])
#     elif num_rna[node][0] == "mrna":
#         for index,node1 in enumerate(sub1):
#             #子图内的邻居节点node,i
#             if new_A[node_dict[node],node_dict[node1]]==1:
#                 if num_rna[node1][0] == "lncrna":
#                     m_lnc.append([node_type_id[node][1],node_type_id[node1][1]])
#                 elif num_rna[node1][0] == "mirna":
#                     m_mi.append([node_type_id[node][1],node_type_id[node1][1]])
#                 elif num_rna[node1][0] == "mrna":
#                     m_m.append([node_type_id[node][1],node_type_id[node1][1]])
# 
# data_lnc_lnc = pd.DataFrame(lnc_lnc).to_csv("../data/Degree0_graph/lnc_lnc.csv",index=False)
# data_lnc_mi = pd.DataFrame(lnc_mi).to_csv("../data/Degree0_graph/lnc_mi.csv",index=False)
# data_lnc_m = pd.DataFrame(lnc_m).to_csv("../data/Degree0_graph/lnc_m.csv",index=False)
# data_mi_lnc = pd.DataFrame(mi_lnc).to_csv("../data/Degree0_graph/mi_lnc.csv",index=False)
# data_mi_mi = pd.DataFrame(mi_mi).to_csv("../data/Degree0_graph/mi_mi.csv",index=False)
# data_mi_m = pd.DataFrame(mi_m).to_csv("../data/Degree0_graph/mi_m.csv",index=False)
# data_m_lnc = pd.DataFrame(m_lnc).to_csv("../data/Degree0_graph/m_lnc.csv",index=False)
# data_m_mi = pd.DataFrame(m_mi).to_csv("../data/Degree0_graph/m_mi.csv",index=False)
# data_m_m = pd.DataFrame(m_m).to_csv("../data/Degree0_graph/m_m.csv",index=False)



In [13]:
#读取各个子图，测试RGCN
subgraphs = pd.read_csv("../data/SubGraph/N400_123.csv")



In [14]:
subgraphs.head()

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
0,2048,2049,2052,2053,2054,2055,2056,2058,2059,2060,...,2033,2034,2036,4087,2040,2041,2042,2043,2045,2047
1,4,5,2055,20,22,24,30,2079,31,32,...,3982,3991,4027,1980,1990,4054,2022,4073,4083,4085
2,2053,6,7,2055,2059,2060,2062,19,2067,2069,...,2029,2030,2031,2033,2034,2035,2036,2040,2042,2043
3,2052,2053,2054,2055,2056,2058,2059,2060,2062,2063,...,2032,2033,2034,2039,2040,2041,2042,2043,2045,2047
4,4097,4105,2058,4109,4111,2066,2069,2070,4117,2072,...,3934,3942,1897,3945,3947,1902,1907,3955,1915,4094


In [15]:
# print(subgraphs.iloc[160])

In [16]:
all_subgraphs = subgraphs.iloc[0:123,:]
print(all_subgraphs.iloc[122,:])

0      2049
1      2051
2      2053
3      2055
4         8
       ... 
395    2039
396    2040
397    2041
398    2042
399    2043
Name: 122, Length: 400, dtype: int64


In [17]:
print(len(all_subgraphs))

123


In [18]:
sub1 = list(all_subgraphs.iloc[0,:])
print(sub1)

[2048, 2049, 2052, 2053, 2054, 2055, 2056, 2058, 2059, 2060, 2062, 2063, 2064, 2065, 2066, 2039, 2067, 2068, 2069, 2070, 24, 2071, 2072, 2073, 2076, 2077, 2078, 2079, 2081, 2082, 2083, 2084, 2085, 2086, 4136, 2088, 2090, 2092, 2094, 2095, 2097, 2098, 2099, 2100, 2102, 2104, 2105, 2106, 2107, 2108, 2109, 4158, 2110, 2113, 2114, 2117, 2119, 2122, 2123, 2124, 2129, 2130, 83, 2132, 2133, 2135, 2139, 2140, 2142, 2143, 2148, 2149, 2151, 2152, 2153, 2157, 2158, 2159, 2161, 2162, 2165, 2167, 2168, 2171, 2172, 2174, 2175, 2176, 2177, 2181, 2183, 2237, 272, 332, 402, 422, 429, 437, 452, 458, 459, 490, 493, 616, 625, 638, 2756, 722, 757, 822, 2961, 2992, 953, 3068, 3069, 3070, 3071, 3073, 3149, 1183, 3248, 3328, 1337, 3423, 3459, 1442, 1452, 1483, 1500, 1502, 1503, 1504, 1506, 1508, 1512, 1518, 1521, 1522, 1523, 1526, 1528, 1529, 1530, 1531, 1535, 1536, 1537, 1538, 3588, 1541, 1542, 1543, 1545, 1548, 1551, 1554, 1555, 1556, 1558, 1559, 1564, 1565, 1566, 1572, 1574, 1575, 1577, 1578, 1580, 1581, 1

In [19]:
# 要把networkx的graph转为异构图
# N80.csv只存了子图节点，还要存子图节点类型，类型和类型之间关系
rna_num = {}
num_rna = {}
curnode = 0
for name in g.ntypes:
    tmp={}
    for i in range(g.number_of_nodes(name)):
        tmp[i]=curnode
        num_rna[curnode]=(name,i)
        curnode+=1
    rna_num[name]=tmp
print(num_rna)

{0: ('lncrna', 0), 1: ('lncrna', 1), 2: ('lncrna', 2), 3: ('lncrna', 3), 4: ('lncrna', 4), 5: ('lncrna', 5), 6: ('lncrna', 6), 7: ('lncrna', 7), 8: ('lncrna', 8), 9: ('lncrna', 9), 10: ('lncrna', 10), 11: ('lncrna', 11), 12: ('lncrna', 12), 13: ('lncrna', 13), 14: ('lncrna', 14), 15: ('lncrna', 15), 16: ('lncrna', 16), 17: ('lncrna', 17), 18: ('lncrna', 18), 19: ('lncrna', 19), 20: ('lncrna', 20), 21: ('lncrna', 21), 22: ('lncrna', 22), 23: ('lncrna', 23), 24: ('lncrna', 24), 25: ('lncrna', 25), 26: ('lncrna', 26), 27: ('lncrna', 27), 28: ('lncrna', 28), 29: ('lncrna', 29), 30: ('lncrna', 30), 31: ('lncrna', 31), 32: ('lncrna', 32), 33: ('lncrna', 33), 34: ('lncrna', 34), 35: ('lncrna', 35), 36: ('lncrna', 36), 37: ('lncrna', 37), 38: ('lncrna', 38), 39: ('lncrna', 39), 40: ('lncrna', 40), 41: ('lncrna', 41), 42: ('lncrna', 42), 43: ('lncrna', 43), 44: ('lncrna', 44), 45: ('lncrna', 45), 46: ('lncrna', 46), 47: ('lncrna', 47), 48: ('lncrna', 48), 49: ('lncrna', 49), 50: ('lncrna', 50),

In [20]:
#开始处理sub1，要知道每个结点属于什么类型，以及节点直接关系属于什么类型
#三种RNA的id，都必须是0-len（rna）
degree0_2_subgraph = {}
path = "N"+str(N)+"/"+str(11)
for k in range(len(all_subgraphs)):
    lnc_lnc = []
    lnc_mi = []
    lnc_m = []
    mi_lnc = []
    mi_mi = []
    mi_m = []
    m_lnc = []
    m_mi = []
    m_m = []
    sub1 = list(all_subgraphs.iloc[k,:])
    rna_num = [0,0,0]
    node_type_id = {}
    lnc_num = 0
    mi_num = 0
    m_num = 0
    
    degree0_2_subgraph[k] = {}
    cursor = 0
    for f,node in enumerate(sub1):
        type = num_rna[node][0]
        degree0_2_subgraph[k][cursor] = node_dict[node]

        if type == "lncrna":
            node_type_id[node] = (type,lnc_num)
            lnc_num += 1
        elif type == "mirna":
            node_type_id[node] = (type,mi_num)
            mi_num += 1
        elif type == "mrna":
            node_type_id[node] = (type,m_num)
            m_num += 1
        cursor += 1
    print(lnc_num,mi_num,m_num,end=" ")
    for index1,node in enumerate(sub1):
        if num_rna[node][0] == "lncrna":
            #判断邻居属性
            for index,node1 in enumerate(sub1):
                #子图内的邻居节点node,i
                if new_A[node_dict[node],node_dict[node1]]==1:
                    if num_rna[node1][0] == "lncrna":
                        lnc_lnc.append([node_type_id[node][1],node_type_id[node1][1]])
                    elif num_rna[node1][0] == "mirna":
                        lnc_mi.append([node_type_id[node][1],node_type_id[node1][1]])
                    elif num_rna[node1][0] == "mrna":
                        lnc_m.append([node_type_id[node][1],node_type_id[node1][1]])
        elif num_rna[node][0] == "mirna":
            for index,node1 in enumerate(sub1):
                #子图内的邻居节点node,i
                if new_A[node_dict[node],node_dict[node1]]==1:
                    if num_rna[node1][0] == "lncrna":
                        mi_lnc.append([node_type_id[node][1],node_type_id[node1][1]])
                    elif num_rna[node1][0] == "mirna":
                        mi_mi.append([node_type_id[node][1],node_type_id[node1][1]])
                    elif num_rna[node1][0] == "mrna":
                        mi_m.append([node_type_id[node][1],node_type_id[node1][1]])
        elif num_rna[node][0] == "mrna":
            for index,node1 in enumerate(sub1):
                #子图内的邻居节点node,i
                if new_A[node_dict[node],node_dict[node1]]==1:
                    if num_rna[node1][0] == "lncrna":
                        m_lnc.append([node_type_id[node][1],node_type_id[node1][1]])
                    elif num_rna[node1][0] == "mirna":
                        m_mi.append([node_type_id[node][1],node_type_id[node1][1]])
                    elif num_rna[node1][0] == "mrna":
                        m_m.append([node_type_id[node][1],node_type_id[node1][1]])
    print(k,len(sub1),len(lnc_lnc),len(lnc_mi),len(lnc_m),len(mi_lnc),len(mi_mi),len(mi_m),len(m_lnc),len(m_mi),len(m_m))
    try:
        data_lnc_lnc = pd.DataFrame(lnc_lnc).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_lnc_lnc.csv",index=False)
        data_lnc_mi = pd.DataFrame(lnc_mi).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_lnc_mi.csv",index=False)
        data_lnc_m = pd.DataFrame(lnc_m).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_lnc_m.csv",index=False)
        data_mi_lnc = pd.DataFrame(mi_lnc).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_mi_lnc.csv",index=False)
        data_mi_mi = pd.DataFrame(mi_mi).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_mi_mi.csv",index=False)
        data_mi_m = pd.DataFrame(mi_m).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_mi_m.csv",index=False)
        data_m_lnc = pd.DataFrame(m_lnc).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_m_lnc.csv",index=False)
        data_m_mi = pd.DataFrame(m_mi).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_m_mi.csv",index=False)
        data_m_m = pd.DataFrame(m_m).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_m_m.csv",index=False)
    except OSError:
        os.mkdir("../data/SubGraph/"+path+"/"+str(k)+"/")
        data_lnc_lnc = pd.DataFrame(lnc_lnc).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_lnc_lnc.csv",index=False)
        data_lnc_mi = pd.DataFrame(lnc_mi).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_lnc_mi.csv",index=False)
        data_lnc_m = pd.DataFrame(lnc_m).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_lnc_m.csv",index=False)
        data_mi_lnc = pd.DataFrame(mi_lnc).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_mi_lnc.csv",index=False)
        data_mi_mi = pd.DataFrame(mi_mi).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_mi_mi.csv",index=False)
        data_mi_m = pd.DataFrame(mi_m).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_mi_m.csv",index=False)
        data_m_lnc = pd.DataFrame(m_lnc).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_m_lnc.csv",index=False)
        data_m_mi = pd.DataFrame(m_mi).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_m_mi.csv",index=False)
        data_m_m = pd.DataFrame(m_m).to_csv("../data/SubGraph/"+path+"/"+str(k)+"/sub_m_m.csv",index=False)


25 64 311 0 400 38 74 195 74 24 1800 195 1800 2012
276 14 110 1 400 7930 18 1034 18 0 288 1034 288 1038
50 93 257 2 400 182 76 339 76 58 2014 339 2014 1356
21 84 295 3 400 2 84 90 84 62 2027 90 2027 1682
3 11 386 4 400 0 4 8 4 2 1115 8 1115 2398
4 11 385 5 400 0 12 1 12 42 1023 1 1023 2120
232 41 127 6 400 8308 57 11035 57 12 384 11035 384 13344
46 125 229 7 400 150 109 386 109 96 1653 386 1653 3010
19 10 371 8 400 0 25 40 25 8 729 40 729 2042
37 44 319 9 400 14 79 92 79 68 1333 92 1333 1348
112 145 143 10 400 518 127 818 127 126 1188 818 1188 2246
8 13 379 11 400 0 17 8 17 2 801 8 801 1394
37 38 325 12 400 62 69 174 69 30 1203 174 1203 1678
163 15 222 13 400 6016 2 6395 2 14 260 6395 260 8064
288 2 110 14 400 7292 1 956 1 0 114 956 114 210
190 2 208 15 400 2404 0 1202 0 0 231 1202 231 816
152 5 243 16 400 1424 11 929 11 12 315 929 315 844
8 5 387 17 400 0 9 10 9 0 506 10 506 2020
138 6 256 18 400 3208 1 4519 1 0 80 4519 80 9154
130 2 268 19 400 2356 0 3637 0 0 53 3637 53 7460
28 32 34

In [21]:
torch.save(degree0_2_subgraph,"degree0_2_subgraph_"+str(N)+"_"+str(11)+"_"+".pt")

In [22]:
aaa = torch.load("degree0_2_subgraph_"+str(N)+"_"+str(11)+"_"+".pt")
a = set([])
state = torch.zeros((3164,1))
for i in range(123):
    for j in range(N):
        state[aaa[i][j]] = 1
        a.add(aaa[i][j])
print(torch.sum(state))
print(len(a))

tensor(3164.)
3164


In [23]:
#读取子图1，然后测试RGCN是否能用这个子图
# path='../data/SubGraph/N300/3/0'
# # a=pd.read_csv(path+'/sub_lnc_lnc.csv')
# print(os.path.getsize(path+'/sub_lnc_lnc.csv'))
# print(os.path.getsize(path+'/sub_m_m.csv'))


In [24]:
# for k in range(282):
#     print(k)
#     g = load_subk_hetero_graph(k)
#     adjacency_matrix=torch.from_numpy(np.array(nx.adjacency_matrix(dgl.to_homogeneous(g).to_networkx().to_undirected()).todense(),dtype=float))
#     sub0 = list(all_subgraphs.iloc[k,:])
#     rna_num = [0,0,0]
#     for node in sub0:
#         type = num_rna[node][0]
#         if type == "lncrna":
#             rna_num[0] = rna_num[0] + 1
#         elif type == "mirna":
#             rna_num[1] = rna_num[1] + 1
#         elif type == "mrna":
#             rna_num[2] = rna_num[2] + 1
#     print(rna_num)
#     eval_net = RGCNDQNModel(32,1,1,['lnc_lnc', 'lnc_mi', 'm_lnc', 'mi_lnc', 'mi_m', 'mi_mi', 'lnc_m', 'm_m','m_mi'],
#                              5,
#                              2)
#     f_lncrna = torch.zeros(rna_num[0], 1)
#     f_mirna = torch.zeros(rna_num[1], 1) # 初始化每个节点的编码特征
#     f_mrna = torch.zeros(rna_num[2], 1)
#     h = {'lncrna': f_lncrna,'mirna': f_mirna, 'mrna': f_mrna}
#     args = []
#     res = eval_net.forward(args,g,rna_num,h)
#     print(res.shape)

In [25]:
# adjacency_matrix=torch.from_numpy(np.array(nx.adjacency_matrix(dgl.to_homogeneous(g).to_networkx().to_undirected()).todense(),dtype=float))

In [26]:
# print(len(adjacency_matrix))

In [27]:
# #需要确定每个子图三种RNA的数量，然后带入
# k=2
# sub0 = list(all_subgraphs.iloc[k,:])
# rna_num = [0,0,0]
# for node in sub0:
#     type = num_rna[node][0]
#     if type == "lncrna":
#         rna_num[0] = rna_num[0] + 1
#     elif type == "mirna":
#         rna_num[1] = rna_num[1] + 1
#     elif type == "mrna":
#         rna_num[2] = rna_num[2] + 1
# print(rna_num)

In [28]:
# eval_net = RGCNDQNModel(32,1,1,['lnc_lnc', 'lnc_mi', 'm_lnc', 'mi_lnc', 'mi_m', 'mi_mi', 'lnc_m', 'm_m','m_mi'],
#                              5,
#                              2)


In [29]:
# f_lncrna = torch.zeros(rna_num[0], 1)
# f_mirna = torch.zeros(rna_num[1], 1) # 初始化每个节点的编码特征
# f_mrna = torch.zeros(rna_num[2], 1)
# h = {'lncrna': f_lncrna,'mirna': f_mirna, 'mrna': f_mrna}
# args = 0
# res = eval_net.forward(args,g,h)

In [30]:
# print(res.shape)

In [31]:
# 再做一个对应关系，degree0 构建一个新图，节点是0-3164
